<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim

## Alumno: Fux, Santiago Javier (CEIA-6ta Cohorte)
### Fecha: 2023-03-22

### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset las siguientes canciones de Patricio Rey y sus redonditos de ricota:

* Alien Duce  
* Aquella solitaria vaca cubana  
* Ñam Fri Frufi Fali Fru  
* Barbazul vs. El Amor Letal  
* Blues de la Artilleria  
* Blues de la Libertad  
* Botija Rapado  
* Buenas Noticias  
* Caña Seca Y Un Membrillo  
* Cancion para naufragios  
* Criminal Mambo  
* Cruz Diablo!  

In [2]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('redondos.txt', sep='/n', header=None)
df.head()

/tmp/ipykernel_768207/3447294478.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('redondos.txt', sep='/n', header=None)


,0
0,Alien duce adorno tu esclavitud
1,y en un edificio en llamas te encanó
2,Nadando en ese minestrón
3,va Porco Rex a Porno Rock!
4,"Si su looz""bola de efectos"" te palmó"


In [3]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 236


### 1 - Preprocesamiento

In [4]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    words = text_to_word_sequence(row[0])
    sentence_tokens.append(words)

2023-03-22 23:23:03.788132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-22 23:23:03.788151: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# Demos un vistazo
sentence_tokens[:2]

[['alien', 'duce', 'adorno', 'tu', 'esclavitud'],
 ['y', 'en', 'un', 'edificio', 'en', 'llamas', 'te', 'encanó']]

### 2 - Crear los vectores (word2vec)

In [6]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [7]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=1,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=1,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [8]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [9]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 236


In [10]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv))

Cantidad de words distintas en el corpus: 501


In [11]:
len(w2v_model.wv.index_to_key)

501

### 3 - Entrenar el modelo generador

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 19174.884765625
Loss after epoch 1: 18879.986328125
Loss after epoch 2: 19769.8984375
Loss after epoch 3: 19154.83203125
Loss after epoch 4: 18638.875
Loss after epoch 5: 17022.5859375
Loss after epoch 6: 13825.2890625
Loss after epoch 7: 10847.5078125
Loss after epoch 8: 8552.15625
Loss after epoch 9: 7540.9375
Loss after epoch 10: 6720.34375
Loss after epoch 11: 6047.359375
Loss after epoch 12: 5827.390625
Loss after epoch 13: 5774.296875
Loss after epoch 14: 5731.09375
Loss after epoch 15: 5588.3125
Loss after epoch 16: 5532.078125
Loss after epoch 17: 5545.21875
Loss after epoch 18: 5307.828125
Loss after epoch 19: 5358.984375
Loss after epoch 20: 5375.296875
Loss after epoch 21: 5175.625
Loss after epoch 22: 5227.015625
Loss after epoch 23: 5334.203125
Loss after epoch 24: 5182.078125
Loss after epoch 25: 5218.953125
Loss after epoch 26: 5362.1875
Loss after epoch 27: 5419.171875
Loss after epoch 28: 5293.765625
Loss after epoch 29: 5368.625
Loss after epoch 30

(88421, 125800)

### 4 - Ensayar

In [13]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["prisión"], topn=3)

[('barbazul', 0.9909407496452332),
 ('aullido', 0.9900803565979004),
 ('letal', 0.9892053604125977)]

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["vaca"], topn=10)

[('aquella', 0.9909690022468567),
 ('nueva', 0.9592849016189575),
 ('hecho', 0.9589110612869263),
 ('cagando', 0.9587357044219971),
 ('zumbador', 0.9584758281707764),
 ('cargados', 0.9584510326385498),
 ('enloquecido', 0.9584219455718994),
 ('breto', 0.9583457708358765),
 ('impalpables', 0.9583221673965454),
 ('asustada', 0.9582536816596985)]

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["negrita"], topn=10)

[('¡vamos', 0.9777770042419434),
 ('vamos', 0.9733973741531372),
 ('bailá', 0.9607186913490295),
 ('hacelo', 0.959125816822052),
 ('hasta', 0.9586584568023682),
 ('mí', 0.9557260870933533),
 ('por', 0.9313434958457947),
 ('fin', 0.927501916885376),
 ('el', 0.8682405948638916),
 ('donde', 0.7844252586364746)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["valija"], topn=5)

[('una', 0.9926223754882812),
 ('nueva', 0.9921937584877014),
 ('mandó', 0.9917711019515991),
 ('poner', 0.9916523098945618),
 ('maldiciones', 0.991549551486969)]

In [17]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["bares"])

[('¡safó', 0.039392612874507904),
 ('rex', -0.6052075028419495),
 ('todo', -0.6321513652801514),
 ('porno', -0.6364663243293762),
 ('fali', -0.6425488591194153),
 ('igual', -0.6555845737457275),
 ('porco', -0.6564461588859558),
 ('fri', -0.6761347055435181),
 ('frufi', -0.688993513584137),
 ('lo', -0.7065088748931885)]

### 5 - Visualizar agrupación de vectores

In [18]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/home/santiago/.local/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/santiago/.local/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


### Conclusiones:
Se observa en el mapa de embeddings el agrupamiento más estrecho entre palabras singulares que aparecen en una misma canción. Podrían mejorarse los resultados incluyendo todas las canciones de la banda.